# Step 3: Building a Vector Database with ChromaDB

This notebook demonstrates the third step in building a RAG (Retrieval-Augmented Generation) pipeline: **creating a vector database from our markdown papers**.

## What is a Vector Database?

A **vector database** stores text as high-dimensional numerical vectors (embeddings) that capture semantic meaning. This enables:
- **Semantic search**: Find content by meaning, not just keywords
- **Fast retrieval**: Approximate nearest neighbor search in milliseconds
- **Scalability**: Handle millions of documents efficiently

## What is ChromaDB?

[ChromaDB](https://www.trychroma.com/) is an open-source embedding database designed for AI applications. It features:
- **Easy to use**: Simple Python API
- **Persistent storage**: Data saved to disk
- **Built-in embeddings**: Or bring your own
- **Fast queries**: Optimized for similarity search

## Overview of This Step

In this notebook, we will:
1. Load markdown files from Step 2
2. Chunk documents into smaller, overlapping pieces
3. Generate embeddings using sentence-transformers
4. Store embeddings in ChromaDB with metadata
5. Test the search functionality

## Why This Matters for RAG

Vector databases are the foundation of RAG systems:
- **Retrieval**: Find relevant context for user queries
- **Context**: Provide LLMs with specific, relevant information
- **Citations**: Track which papers chunks come from
- **Efficiency**: Search thousands of documents in milliseconds

### The RAG Pipeline So Far

1. **Step 1**: ✅ Collected papers from OpenReview
2. **Step 2**: ✅ Converted PDFs to structured Markdown
3. **Step 3**: 🎯 **Build vector database** ← We are here
4. **Next Steps**: Query system and answer generation

## Import Required Libraries

In [ ]:
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/4

In [ ]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = Path("/content/drive/MyDrive/RAG")
PDF_FOLDER = BASE_PATH / "block1_output_pdfs"

MD_FOLDER = BASE_PATH / "block2_output_markdowns"


if PDF_FOLDER.exists() and MD_FOLDER.exists():
    print("Ready for Block3!")
else:
    print(f"Rerun previous codes")


for file in MD_FOLDER.iterdir():
    print(file.name)



PERSIST_FOLDER = BASE_PATH / "block3_output_database"

PERSIST_FOLDER.mkdir(parents=True, exist_ok=True)

print(f"PDF Folder: {PDF_FOLDER}")
print(f"Markdown Folder: {MD_FOLDER}")
print(f"DB Folder: {PERSIST_FOLDER}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ready for Block3!
14296.md
14294.md
14293.md
14290.md
14287.md
PDF Folder: /content/drive/MyDrive/RAG/block1_output_pdfs
Markdown Folder: /content/drive/MyDrive/RAG/block2_output_markdowns
DB Folder: /content/drive/MyDrive/RAG/block3_output_database


In [ ]:
import os
import re
from pathlib import Path
from typing import List, Dict
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Understanding Document Chunking

### Why Chunk Documents?

Academic papers are long (10-30 pages), but:
- **LLM context limits**: Can't process entire papers at once
- **Retrieval precision**: Smaller chunks = more specific matches
- **Better relevance**: Return only the most relevant sections

### Chunking Strategy

Our approach uses:
1. **Fixed size chunks**: ~1000 characters per chunk
2. **Overlap**: 200 characters overlap between chunks
3. **Smart boundaries**: Break at sentences/paragraphs, not mid-word
4. **Metadata**: Track which paper and chunk number

### Key Parameters

- **chunk_size**: Target characters per chunk (default: 1000)
- **chunk_overlap**: Characters shared between consecutive chunks (default: 200)
- **Overlap benefits**: Ensures context isn't lost at boundaries

## Define the Document Chunker Class

This class handles:
- Cleaning OCR artifacts from Step 2
- Splitting text into overlapping chunks
- Breaking at natural boundaries (sentences, paragraphs)
- Preserving metadata for each chunk

In [ ]:
class DocumentChunker:
    """Split documents into chunks with overlap for better retrieval."""

    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        """
        Args:
            chunk_size: Target number of characters per chunk
            chunk_overlap: Number of characters to overlap between chunks
        """
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def clean_markdown(self, text: str) -> str:
        """Remove OCR artifacts and clean up markdown."""
        # Remove OCR tags like <|ref|>title<|/ref|><|det|>[[175, 99, 825, 146]]<|/det|>
        text = re.sub(r'<\|ref\|>.*?<\/ref\|><\|det\|>.*?<\/det\|>', '', text)

        # Remove page markers
        text = re.sub(r'--- Page \d+ ---', '', text)

        # Clean up extra whitespace
        text = re.sub(r'\n\s*\n\s*\n', '\n\n', text)
        text = text.strip()

        return text

    def chunk_text(self, text: str, metadata: Dict) -> List[Dict]:
        """
        Split text into overlapping chunks.

        Args:
            text: Document text to chunk
            metadata: Metadata to attach to each chunk

        Returns:
            List of dicts with 'text', 'metadata', and 'chunk_id'
        """
        text = self.clean_markdown(text)

        chunks = []
        start = 0
        chunk_num = 0

        while start < len(text):
            # Find the end of this chunk
            end = start + self.chunk_size

            # If not at the end, try to break at a sentence or paragraph
            if end < len(text):
                # Look for paragraph break first
                para_break = text.rfind('\n\n', start, end)
                if para_break > start + self.chunk_size // 2:
                    end = para_break
                else:
                    # Look for sentence break
                    sentence_break = max(
                        text.rfind('. ', start, end),
                        text.rfind('.\n', start, end),
                        text.rfind('! ', start, end),
                        text.rfind('? ', start, end)
                    )
                    if sentence_break > start + self.chunk_size // 2:
                        end = sentence_break + 1

            chunk_text = text[start:end].strip()

            if chunk_text:
                chunk_metadata = metadata.copy()
                chunk_metadata['chunk_num'] = chunk_num

                chunks.append({
                    'text': chunk_text,
                    'metadata': chunk_metadata,
                    'chunk_id': f"{metadata['paper_id']}_chunk_{chunk_num}"
                })

                chunk_num += 1

            # Move to next chunk with overlap
            start = end - self.chunk_overlap
            if start < 0:
                start = end

        return chunks

print("✓ DocumentChunker class defined")

✓ DocumentChunker class defined


## Understanding Embeddings

### What are Embeddings?

Embeddings convert text into numerical vectors that capture meaning:
- **Vectors**: Arrays of numbers (e.g., 384 or 768 dimensions)
- **Semantic similarity**: Similar meanings → similar vectors
- **Distance metrics**: Cosine similarity, Euclidean distance

### Sentence Transformers

We use [sentence-transformers](https://www.sbert.net/) which provides:
- **Pre-trained models**: Ready to use, no training needed
- **Fast encoding**: Optimized for CPU and GPU
- **Good quality**: State-of-the-art performance

### Model: all-MiniLM-L6-v2

This model is ideal for our use case:
- **Size**: 80MB (lightweight)
- **Speed**: Fast encoding (~14K sentences/sec on GPU)
- **Quality**: Good balance of performance and efficiency
- **Dimensions**: 384-dimensional vectors
- **Use case**: Perfect for semantic search and retrieval

## Define the Vector Database Class

This class manages our ChromaDB collection:
- Initializes the database and embedding model
- Adds papers by chunking and embedding them
- Provides search functionality
- Tracks statistics

In [ ]:
class PaperVectorDB:
    """Manage vector database for academic papers."""

    def __init__(self,
                 collection_name: str = "iclr_papers",
                 persist_directory: Path = PERSIST_FOLDER,
                 embedding_model: str = "all-MiniLM-L6-v2"):
        """
        Initialize the vector database.

        Args:
            collection_name: Name for the ChromaDB collection
            persist_directory: Directory to persist the database
            embedding_model: SentenceTransformer model name
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory

        # Initialize ChromaDB client
        self.client = chromadb.PersistentClient(
            path=persist_directory,
            settings=Settings(anonymized_telemetry=False)
        )

        # Initialize embedding model
        print(f"Loading embedding model: {embedding_model}")
        self.embedding_model = SentenceTransformer(embedding_model)

        # Create or get collection
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            metadata={"description": "ICLR 2025 papers"}
        )

        print(f"Collection '{collection_name}' initialized with {self.collection.count()} documents")

    def add_papers(self, papers_dir: str, chunk_size: int = 1000, chunk_overlap: int = 200):
        """
        Add papers from a directory to the vector database.

        Args:
            papers_dir: Directory containing markdown files
            chunk_size: Size of text chunks
            chunk_overlap: Overlap between chunks
        """
        papers_path = Path(papers_dir)
        md_files = list(papers_path.glob("*.md"))

        if not md_files:
            print(f"No markdown files found in {papers_dir}")
            return

        print(f"\nFound {len(md_files)} markdown files")

        chunker = DocumentChunker(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        all_chunks = []
        for md_file in md_files:
            print(f"Processing: {md_file.name}")

            # Read the file
            with open(md_file, 'r', encoding='utf-8') as f:
                text = f.read()

            # Create metadata
            paper_id = md_file.stem
            metadata = {
                'paper_id': paper_id,
                'filename': md_file.name,
                'source': 'ICLR 2025'
            }

            # Chunk the document
            chunks = chunker.chunk_text(text, metadata)
            all_chunks.extend(chunks)
            print(f"  Created {len(chunks)} chunks")

        if not all_chunks:
            print("No chunks created!")
            return

        print(f"\nTotal chunks: {len(all_chunks)}")
        print("Generating embeddings...")

        # Generate embeddings
        texts = [chunk['text'] for chunk in all_chunks]
        embeddings = self.embedding_model.encode(
            texts,
            show_progress_bar=True,
            convert_to_numpy=True
        )

        print("Adding to ChromaDB...")

        # Add to ChromaDB
        self.collection.add(
            ids=[chunk['chunk_id'] for chunk in all_chunks],
            embeddings=embeddings.tolist(),
            documents=texts,
            metadatas=[chunk['metadata'] for chunk in all_chunks]
        )

        print(f"✅ Successfully added {len(all_chunks)} chunks to the database")
        print(f"Total documents in collection: {self.collection.count()}")

    def search(self, query: str, n_results: int = 5) -> Dict:
        """
        Search for relevant chunks.

        Args:
            query: Search query
            n_results: Number of results to return

        Returns:
            Dictionary with results
        """
        # Generate query embedding
        query_embedding = self.embedding_model.encode(query).tolist()

        # Query ChromaDB
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results
        )

        return results

    def get_stats(self) -> Dict:
        """Get statistics about the collection."""
        count = self.collection.count()

        # Get a sample to see what papers we have
        if count > 0:
            sample = self.collection.get(limit=10)
            papers = set(meta.get('paper_id', 'unknown') for meta in sample['metadatas'])
        else:
            papers = set()

        return {
            'total_chunks': count,
            'sample_papers': list(papers)
        }

print("✓ PaperVectorDB class defined")

✓ PaperVectorDB class defined


## Configure Parameters

Set the parameters for building our vector database:
- **Papers directory**: Location of markdown files from Step 2
- **Database directory**: Where to persist ChromaDB
- **Chunk size**: Characters per chunk
- **Chunk overlap**: Shared characters between chunks

In [ ]:
# Configuration
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

print("="*60)
print("Building Vector Database for RAG System")
print("="*60)
print(f"\n📋 Configuration:")
print(f"   Markdown directory: {MD_FOLDER}")
print(f"   Database directory: {PERSIST_FOLDER}")
print(f"   Chunk size: {CHUNK_SIZE} characters")
print(f"   Chunk overlap: {CHUNK_OVERLAP} characters")

Building Vector Database for RAG System

📋 Configuration:
   Markdown directory: /content/drive/MyDrive/RAG/block2_output_markdowns
   Database directory: /content/drive/MyDrive/RAG/block3_output_database
   Chunk size: 1000 characters
   Chunk overlap: 200 characters


## Initialize the Vector Database

This will:
1. Create a ChromaDB client with persistent storage
2. Load the sentence-transformer embedding model
3. Create or load the collection

**Note**: First-time initialization will download the embedding model (~80MB). This may take a minute.

In [ ]:
# Initialize vector DB
vectordb = PaperVectorDB(
    collection_name="iclr_papers",
    persist_directory=PERSIST_FOLDER,
    embedding_model="all-MiniLM-L6-v2"
)

Loading embedding model: all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Collection 'iclr_papers' initialized with 0 documents


## Add Papers to the Database

This is the main processing step:
1. **Load markdown files** from the papers directory
2. **Chunk each document** into ~1000 character pieces with overlap
3. **Generate embeddings** for all chunks using the sentence-transformer model
4. **Store in ChromaDB** with metadata (paper ID, chunk number, source)

**Note**: This may take a few minutes depending on the number and size of papers. The progress bar shows embedding generation progress.

In [ ]:
# Add papers to the database
vectordb.add_papers(
    papers_dir=MD_FOLDER,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)


Found 5 markdown files
Processing: 14296.md
  Created 69 chunks
Processing: 14294.md
  Created 63 chunks
Processing: 14293.md
  Created 88 chunks
Processing: 14290.md
  Created 129 chunks
Processing: 14287.md
  Created 322 chunks

Total chunks: 671
Generating embeddings...


Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Adding to ChromaDB...
✅ Successfully added 671 chunks to the database
Total documents in collection: 671


## View Database Statistics

Let's examine what we've built:

In [ ]:
# Show statistics
stats = vectordb.get_stats()

print(f"\n{'='*60}")
print("Database Statistics:")
print(f"{'='*60}")
print(f"  Total chunks: {stats['total_chunks']}")
print(f"  Papers in database: {len(stats['sample_papers'])}")
print(f"\n  Sample paper IDs: {', '.join(stats['sample_papers'][:5])}")
if len(stats['sample_papers']) > 5:
    print(f"  ... and {len(stats['sample_papers']) - 5} more")
print(f"{'='*60}")


Database Statistics:
  Total chunks: 671
  Papers in database: 1

  Sample paper IDs: 14296


## Test the Search Functionality

Let's verify that our vector database works by performing some test searches. We'll query the database with different questions and see what it retrieves.

### How Search Works

1. **Query embedding**: Convert the question to a vector
2. **Similarity search**: Find chunks with similar vectors (cosine similarity)
3. **Rank results**: Return top-k most similar chunks
4. **Metadata**: Include paper ID and chunk information

In [ ]:
print("\n" + "="*60)
print("Testing Search Functionality")
print("="*60)

# Test Query 1: Technical question
test_query_1 = "What is sparse training in neural networks?"
print(f"\n🔍 Query 1: {test_query_1}")

results = vectordb.search(test_query_1, n_results=3)

print("\n📊 Top 3 Results:")
for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
    print(f"\n{i}. Paper: {metadata['paper_id']} (Chunk {metadata['chunk_num']})")
    print(f"   Source: {metadata['source']}")
    print(f"   Preview: {doc[:200]}...")


Testing Search Functionality

🔍 Query 1: What is sparse training in neural networks?

📊 Top 3 Results:

1. Paper: 14293 (Chunk 11)
   Source: ICLR 2025
   Preview: ared to traditional adapter methods, while achieving superior performance on GLUE.  

<|ref|>sub_title<|/ref|><|det|>[[174, 544, 577, 559]]<|/det|>
### 2.2 IMPLICIT REGULARIZATION IN NEURAL NETWORKS  ...

2. Paper: 14293 (Chunk 13)
   Source: ICLR 2025
   Preview: ads to learning less and forgetting less compared to full fine- tuning, suggesting it constrains models in ways that mitigate catastrophic forgetting and may promote positive transfer between tasks.  ...

3. Paper: 14293 (Chunk 12)
   Source: ICLR 2025
   Preview: s gained attention in deep learning. Implicit regularization happens without explicit regularization terms, as seen in several training dynamics and optimization algorithms (Gunasekar et al., 2017).  ...


In [ ]:
# Test Query 2: Different topic
test_query_2 = "How do transformers handle attention mechanisms?"
print(f"\n🔍 Query 2: {test_query_2}")

results = vectordb.search(test_query_2, n_results=3)

print("\n📊 Top 3 Results:")
for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
    print(f"\n{i}. Paper: {metadata['paper_id']} (Chunk {metadata['chunk_num']})")
    print(f"   Source: {metadata['source']}")
    print(f"   Preview: {doc[:200]}...")


🔍 Query 2: How do transformers handle attention mechanisms?

📊 Top 3 Results:

1. Paper: 14293 (Chunk 17)
   Source: ICLR 2025
   Preview: - th instance of the \(j\) - th task, and \(M_{j}\) indicates the total number of samples in task \(j\) .  

<|ref|>sub_title<|/ref|><|det|>[[173, 216, 383, 230]]<|/det|>
### 3.1 MODEL ARCHITECTURE  
...

2. Paper: 14293 (Chunk 32)
   Source: ICLR 2025
   Preview: 5 model \(M_{\theta}\) , we replace specific linear layers in the self- attention and cross- attention modules with our UnoLoRA layers. The replacement occurs in both the encoder and decoder blocks:  ...

3. Paper: 14293 (Chunk 72)
   Source: ICLR 2025
   Preview: 824, 493]]<|/det|>
Behnam Neyshabur, Srinadh Bhojanapalli, David McAllester, and Nati Srebro. Exploring generalization in deep learning. Advances in neural information processing systems, 30, 2017.  
...


In [ ]:
# Test Query 3: Methodological question
test_query_3 = "What evaluation metrics are used for model performance?"
print(f"\n🔍 Query 3: {test_query_3}")

results = vectordb.search(test_query_3, n_results=3)

print("\n📊 Top 3 Results:")
for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
    print(f"\n{i}. Paper: {metadata['paper_id']} (Chunk {metadata['chunk_num']})")
    print(f"   Source: {metadata['source']}")
    print(f"   Preview: {doc[:200]}...")

print("\n" + "="*60)
print("✅ Vector database built and tested successfully!")
print(f"   Location: {PERSIST_DIR}")
print("="*60)


🔍 Query 3: What evaluation metrics are used for model performance?

📊 Top 3 Results:

1. Paper: 14287 (Chunk 27)
   Source: ICLR 2025
   Preview: benchmarks. We find EXE shows similar relative model performance between models as seen in coding benchmarks such as HumanEval (Chen et al., 2021) and as seen in benchmarks requiring logical inference...

2. Paper: 14287 (Chunk 17)
   Source: ICLR 2025
   Preview: l comparison. See detailed methodology in Appendix A.7 and see examples of generated outputs in Appendix A.1.  

<|ref|>sub_title<|/ref|><|det|>[[174, 747, 348, 761]]<|/det|>
### 2.2 FEATURES OF EXE  ...

3. Paper: 14287 (Chunk 0)
   Source: ICLR 2025
   Preview: <|ref|>title<|/ref|><|det|>[[173, 100, 821, 146]]<|/det|>
# EXECUTION-EVAL: CAN LANGUAGE MODELS EXECUTE REAL-WORLD CODE?  

<|ref|>text<|/ref|><|det|>[[183, 171, 398, 198]]<|/det|>
Anonymous authors P...

✅ Vector database built and tested successfully!
   Location: ./block3_output_database


## Summary and Key Takeaways

### What We Accomplished

In this notebook, we successfully:

1. ✅ **Loaded markdown papers** from Step 2
2. ✅ **Chunked documents** into ~1000 character pieces with overlap
3. ✅ **Generated embeddings** using sentence-transformers
4. ✅ **Built a vector database** with ChromaDB
5. ✅ **Tested search** with semantic queries

### Files Created

- **`block3_output_database/`**: ChromaDB persistent storage
  - `chroma.sqlite3`: Database file
  - Collection data with embeddings and metadata

### Database Characteristics

Our vector database now contains:
- **Semantic search**: Find content by meaning, not keywords
- **Fast retrieval**: Millisecond query times
- **Metadata tracking**: Paper IDs, chunk numbers, source info
- **Persistent storage**: Data saved to disk for reuse

### Quality Indicators

A good vector database should:
- ✓ Return relevant chunks for queries
- ✓ Maintain chunk context with overlap
- ✓ Preserve metadata for citation
- ✓ Handle diverse query types

## Next Step: Query and Answer Generation

### What's Next in the RAG Pipeline?

Now that we have a vector database, the final step is to **build the query interface and answer generation system**. This involves:

#### 1. Query Processing

Accept user questions and prepare them for retrieval:
- **Input validation**: Clean and format queries
- **Query expansion**: Add context or rephrase if needed
- **Embedding generation**: Convert query to vector

#### 2. Retrieval

Find the most relevant chunks from our database:
- **Semantic search**: Use query embedding to find similar chunks
- **Ranking**: Order results by relevance score
- **Filtering**: Apply metadata filters (e.g., specific papers)
- **Context window**: Retrieve enough context for LLM

#### 3. Answer Generation

Use an LLM to generate answers based on retrieved context:
- **Prompt engineering**: Design effective prompts
- **Context injection**: Provide relevant chunks to LLM
- **Answer synthesis**: Generate coherent responses
- **Citation**: Reference source papers and chunks

#### 4. Response Formatting

Present answers with proper attribution:
- **Answer text**: Clear, concise response
- **Citations**: Link to source papers and sections
- **Confidence**: Indicate reliability
- **Follow-up**: Suggest related queries

### The Complete RAG Pipeline

1. **Step 1**: ✅ Collected papers from OpenReview
2. **Step 2**: ✅ Converted PDFs to structured Markdown
3. **Step 3**: ✅ Built vector database with ChromaDB
4. **Next**: Prompt + Embedding + Query -> Answer

### Key Advantages of Our Approach

Our RAG system will provide:
1. **Accurate answers**: Grounded in actual paper content
2. **Proper citations**: Track which papers support each answer
3. **Up-to-date knowledge**: Based on recent conference papers
4. **Explainable**: Show retrieved chunks used for answers
5. **Fast**: Sub-second retrieval times

### Typical RAG System Metrics

A well-built RAG system should achieve:
- **Retrieval Precision**: 70-90% of retrieved chunks are relevant
- **Answer Accuracy**: High factual correctness
- **Citation Quality**: Proper attribution to sources
- **Response Time**: < 2 seconds end-to-end
- **User Satisfaction**: Helpful, well-cited answers

### RAG vs. Pure LLM

| Aspect | Pure LLM | RAG System |
|--------|----------|------------|
| Knowledge cutoff | Fixed training date | Always current |
| Citations | Cannot cite | Provides sources |
| Accuracy | May hallucinate | Grounded in data |
| Domain specific | Limited | Excellent |
| Transparency | Black box | Explainable |

